In [1]:
import pandas as pd
import pyodbc as pc
from sqlalchemy import create_engine
import datetime as datetime
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',5000)

In [2]:
databaseName = 'sreeman_live'
username = 'sa'
password = 'Hemanth@123'
server = 'DESKTOP-L57QDTP\SQLEXPRESS'
driver= '{SQL Server Native Client 10.0}'
CONNECTION_STRING = 'DRIVER='+driver+';SERVER='+server+';DATABASE='+databaseName+';UID='+username+';PWD='+ password
#Create/Open a Connection to Microsoft's SQL Server
conn = pc.connect(CONNECTION_STRING)
#Close the Connection
#conn.close()

databaseName1 = 'suriya_live'
username1 = 'sa'
password1 = 'Hemanth@123'
server1 = 'DESKTOP-L57QDTP\SQLEXPRESS'
driver1= '{SQL Server Native Client 10.0}'

CONNECTION_STRING2 = 'DRIVER='+driver1+';SERVER='+server1+';DATABASE='+databaseName1+';UID='+username1+';PWD='+ password1
#Create/Open a Connection to Microsoft's SQL Server
conn1 = pc.connect(CONNECTION_STRING2)
#Close the Connection
#conn.close()

sql = "SELECT * FROM DBO.Sreeman_full_data_view"
df_sreem = pd.read_sql(sql,conn)

sql1 = "SELECT * FROM DBO.Suriya_full_data_view"
df_suri = pd.read_sql(sql1,conn1)

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:15: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:15: SyntaxWarning: invalid escape sequence '\S'
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:4: SyntaxWarning: invalid escape sequence '\S'
  server = 'DESKTOP-L57QDTP\SQLEXPRESS'
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:15: SyntaxWarning: invalid escape sequence '\S'
  server1 = 'DESKTOP-L57QDTP\SQLEXPRESS'
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sreem = pd.read_sql(sql,conn)
C:\Users\new\AppData\Local\Temp\ipykernel_6968\3544130938.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Othe

In [ ]:
# Set up the connections
databaseName = 'sreeman_live'
username = 'sa'
password = 'sa'
server = 'SREEMAN-NEW'
driver = '{SQL Server Native Client 10.0}'
CONNECTION_STRING = 'DRIVER=' + driver + ';SERVER=' + server + ';DATABASE=' + databaseName + ';UID=' + username + ';PWD=' + password

databaseName1 = 'suriya_live'
username1 = 'sa'
password1 = 'sa'
server1 = 'SREEMAN-NEW'
driver1 = '{SQL Server Native Client 10.0}'
CONNECTION_STRING2 = 'DRIVER=' + driver1 + ';SERVER=' + server1 + ';DATABASE=' + databaseName1 + ';UID=' + username1 + ';PWD=' + password1

# Create/Open connections
conn = pyodbc.connect(CONNECTION_STRING)
conn1 = pyodbc.connect(CONNECTION_STRING2)

# Execute SQL queries and create DataFrames
sql = "SELECT * FROM DBO.Sreeman_full_data_view"
df_sreem = pd.read_sql(sql, conn)

sql1 = "SELECT * FROM DBO.Suriya_full_data_view"
df_suri = pd.read_sql(sql1, conn1)

In [3]:
#df_sreeman=df_sreeman
df_suriya=df_suri
df_sreeman=df_sreem

df_sreeman['Company']='Sreeman'
df_suriya['Company']='Suriya'

df = pd.concat([df_sreeman, df_suriya], ignore_index=True)


def date_ext_mon1(mon):
    datee = datetime.datetime.strptime(mon, "%Y-%m-%d")
    month = datetime.date(datee.year, datee.month, datee.day).strftime('%B')
    return month

def date_ext_year1(year):
    #print(year)
    datee = datetime.datetime.strptime(year, "%Y-%m-%d")
    Year = datee.year
    return Year

def date_ext_day1(day):
    datee = datetime.datetime.strptime(day, "%Y-%m-%d")
    Day = datee.day
    return Day

#this is for sreeman.
##this is to filter out the unit wise payment.
df['lr_hire_date']= df['lr_hire_date'].astype(str)
df['lr_hire_date']= df['lr_hire_date'].astype(str)
df['YEAR']=df['lr_hire_date'].apply(lambda x: date_ext_year1(x))
df['MONTH']=df['lr_hire_date'].apply(lambda x: date_ext_mon1(x))
df['DAY']=df['lr_hire_date'].apply(lambda x: date_ext_day1(x))

In [32]:
df1=df

In [33]:
df1=df1[(df1['lr_hire_date']>='2023-12-01') & (df1['lr_hire_date']<='2023-12-09')]

In [34]:
shipped_from='M/S. AQUASUB ENGINEERING'
freight_billed_to='M/S. AQUASUB ENGINEERING (VIJ)'
# dont use this while generating report for invoice for now
df1=df1[(df1['from_branch']==shipped_from) & (df1['to_branch']==freight_billed_to)]

In [35]:
df2=df1.groupby(['Company','gdm_no','lr_hire_date','lr_number','lorry_reg_number','LR_party_invoice','from_branch','to_branch','LR_no_of_packages','LR_weight','consignor_addr','from_city','consignor_pincode','consignor_state','consignee_addr','to_city','consignee_pincode','consignee_state','consignee_gstin','consignor_gstin'])['LR_Rate'].unique().reset_index().explode('LR_Rate')
df3=df2.groupby(['Company','gdm_no','lr_hire_date','lr_number','lorry_reg_number','from_branch','to_branch','LR_weight','LR_Rate','consignor_addr','from_city','consignor_pincode','consignor_state','consignee_addr','to_city','consignee_pincode','consignee_state','consignee_gstin','consignor_gstin']).agg({'LR_no_of_packages':'sum'}).reset_index()
df4=df3[['Company','gdm_no','from_branch','to_branch','lr_hire_date','lr_number','lorry_reg_number','LR_no_of_packages','LR_weight','LR_Rate','consignor_addr','from_city','consignor_pincode','consignor_state','consignee_addr','to_city','consignee_pincode','consignee_state','consignee_gstin','consignor_gstin']]
df4['stat_chg']=15
df4['total_amount']=round((df4['LR_weight']*df4['LR_Rate'])+df4['stat_chg'])
#df4=df4[['Company','gdm_no','from_branch','to_branch','lr_hire_date', 'lr_number', 'lorry_reg_number', 'LR_no_of_packages', 'LR_weight', 'LR_Rate','stat_chg', 'total_amount']]

C:\Users\new\AppData\Local\Temp\ipykernel_6968\844300457.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['stat_chg']=15
C:\Users\new\AppData\Local\Temp\ipykernel_6968\844300457.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['total_amount']=round((df4['LR_weight']*df4['LR_Rate'])+df4['stat_chg'])


In [21]:
df4.to_excel('taxinvoicereport.xlsx')

In [39]:
df4['consignee_gstin'].unique()[0]
df4['consignor_gstin'].unique()[0]

'37AADFA8028P1Z1'

In [41]:
df4

,Company,gdm_no,from_branch,to_branch,lr_hire_date,lr_number,lorry_reg_number,LR_no_of_packages,LR_weight,LR_Rate,consignor_addr,from_city,consignor_pincode,consignor_state,consignee_addr,to_city,consignee_pincode,consignee_state,consignee_gstin,consignor_gstin,stat_chg,total_amount
0,Sreeman,37244,M/S. AQUASUB ENGINEERING,M/S. AQUASUB ENGINEERING (VIJ),2023-12-01,195989,TN52H3935,219,10897.0,2.32,"THUDIYALUR POST, METTUPALAYAM ROAD,",Coimbatore,641034,Tamil Nadu,"75,76 AND 77 BLOCK NO. 05 NEW AUTO NAGAR KANUR...",Vijayawada,520007,Andhra Pradesh,37AADFA8028P1Z1,33AADFA8028P1Z9,15,25296.0
1,Sreeman,37250,M/S. AQUASUB ENGINEERING,M/S. AQUASUB ENGINEERING (VIJ),2023-12-04,196016,TN28BY1188,142,6577.0,2.32,"THUDIYALUR POST, METTUPALAYAM ROAD,",Coimbatore,641034,Tamil Nadu,"75,76 AND 77 BLOCK NO. 05 NEW AUTO NAGAR KANUR...",Vijayawada,520007,Andhra Pradesh,37AADFA8028P1Z1,33AADFA8028P1Z9,15,15274.0
2,Sreeman,37254,M/S. AQUASUB ENGINEERING,M/S. AQUASUB ENGINEERING (VIJ),2023-12-06,196032,TN52AA4821,248,14003.0,2.32,"THUDIYALUR POST, METTUPALAYAM ROAD,",Coimbatore,641034,Tamil Nadu,"75,76 AND 77 BLOCK NO. 05 NEW AUTO NAGAR KANUR...",Vijayawada,520007,Andhra Pradesh,37AADFA8028P1Z1,33AADFA8028P1Z9,15,32502.0
3,Sreeman,37257,M/S. AQUASUB ENGINEERING,M/S. AQUASUB ENGINEERING (VIJ),2023-12-07,196047,TN52P9157,174,9550.0,2.32,"THUDIYALUR POST, METTUPALAYAM ROAD,",Coimbatore,641034,Tamil Nadu,"75,76 AND 77 BLOCK NO. 05 NEW AUTO NAGAR KANUR...",Vijayawada,520007,Andhra Pradesh,37AADFA8028P1Z1,33AADFA8028P1Z9,15,22171.0
4,Sreeman,37261,M/S. AQUASUB ENGINEERING,M/S. AQUASUB ENGINEERING (VIJ),2023-12-09,196064,TN34W5060,286,13921.0,2.32,"THUDIYALUR POST, METTUPALAYAM ROAD,",Coimbatore,641034,Tamil Nadu,"75,76 AND 77 BLOCK NO. 05 NEW AUTO NAGAR KANUR...",Vijayawada,520007,Andhra Pradesh,37AADFA8028P1Z1,33AADFA8028P1Z9,15,32312.0


In [24]:
consignor_addr=df4['consignor_addr'].unique()[0]
consignor_pincode=df4['consignor_pincode'].unique()[0]
consignor_state=df4['consignor_state'].unique()[0]
from_city=df4['from_city'].unique()[0]


In [9]:
consignee_addr=df4['consignee_addr'].unique()[0]
consignee_pincode=df4['consignee_pincode'].unique()[0]
consignee_state=df4['consignee_state'].unique()[0]
to_city=df4['to_city'].unique()[0]


In [10]:
consignor_addr_combined=consignor_addr.lower()+" "+from_city.upper()+" "+consignor_pincode
consignee_addr_combined=consignee_addr.lower()+" "+to_city.upper()+" "+consignee_pincode

In [11]:
LR_no_of_packages=df4['LR_no_of_packages'].sum()
LR_weight=df4['LR_weight'].sum()
total_amount=df4['total_amount'].sum()

In [12]:
import inflect

In [13]:
total_amount=33186
def number_to_words(number):
    p = inflect.engine()
    words = p.number_to_words(number,andword="and")
    words = words.replace(',', '')  # Remove the comma
    return words

words = number_to_words(total_amount)
word_expand=words.capitalize() + " only"


In [14]:
word_expand


'Thirty-three thousand one hundred and eighty-six only'

In [15]:
CGST_RATE=2.5%
SGST_RATE=2.5%
IGST_RATE=5%

SyntaxError: invalid syntax (1698275974.py, line 1)

In [16]:
total_amount=2930
CGST_AMOUNT=(2.5/100)*total_amount
CGST_AMOUNT=round(CGST_AMOUNT,2)
SGST_AMOUNT=(2.5/100)*total_amount
SGST_AMOUNT=round(CGST_AMOUNT,2)
TOTAL_AMOUNT_GST=CGST_AMOUNT+SGST_AMOUNT


In [17]:
IGST_AMOUNT=(5/100)*total_amount
IGST_AMOUNT=IGST_AMOUNT

In [18]:
SGST_AMOUNT,SGST_AMOUNT,TOTAL_AMOUNT_GST,IGST_AMOUNT

(73.25, 73.25, 146.5, 146.5)

In [19]:
round(SGST_AMOUNT,2)

73.25